In [2]:
%config InlineBackend.figure_format = 'retina'
import json
import nltk
nltk.download('stopwords')
from nltk.tokenize import TweetTokenizer
tknzr = TweetTokenizer()
from nltk.util import ngrams
from textblob import TextBlob
import re

import advertools as adv
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import cufflinks as cf
pd.set_option('display.max_columns', None)
from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import *
import wordcloud

init_notebook_mode(connected=True)
cf.go_offline()

[nltk_data] Downloading package stopwords to C:\Users\deepak
[nltk_data]     patel\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


SET UP TWITTER CONNECTION

In [3]:
app_key = '-------------------'
app_secret = '-------------------------------------------'
oauth_token = '------------------------------------------------'
oauth_token_secret = '--------------------------------------------'



auth_params = {
    'app_key': app_key,
    'app_secret': app_secret,
    'oauth_token': oauth_token,
    'oauth_token_secret': oauth_token_secret,
}
adv.twitter.set_auth_params(**auth_params)

PULL IN TWEEETS USING VARIOUS HASHTAGS

In [4]:
# Pull in tweet from twitter
got = adv.twitter.search(q='#gameofthrones -filter:retweets', lang='en', count=3000, tweet_mode='extended', since='2019-04-01', until='2019-04-27')
# got2 = adv.twitter.search(q='#gameofthrones -filter:retweets', lang='en', count=3000, tweet_mode='extended', since='2019-04-03', until='2019-04-04')
# got3 = adv.twitter.search(q='#gameofthrones -filter:retweets', lang='en', count=3000, tweet_mode='extended', since='2019-04-05', until='2019-04-06')
# got4 = adv.twitter.search(q='#gameofthrones -filter:retweets', lang='en', count=3000, tweet_mode='extended', since='2019-04-06', until='2019-04-07')
# got5 = adv.twitter.search(q='#gameofthrones -filter:retweets', lang='en', count=3000, tweet_mode='extended', since='2019-04-08', until='2019-04-09')
# got6 = adv.twitter.search(q='#gameofthrones -filter:retweets', lang='en', count=3000, tweet_mode='extended', since='2019-04-09', until='2019-04-10')


2019-04-28 00:38:01,046 | INFO | twitter.py:163 | wrapper | search | Requesting: count=100, max_id=None, q=#gameofthrones -filter:retweets, lang=en, tweet_mode=extended, since=2019-04-01, until=2019-04-27
2019-04-28 00:38:01,927 | INFO | twitter.py:163 | wrapper | search | Requesting: count=100, max_id=1121922459672961024, q=#gameofthrones -filter:retweets, lang=en, tweet_mode=extended, since=2019-04-01, until=2019-04-27
2019-04-28 00:38:02,371 | INFO | twitter.py:163 | wrapper | search | Requesting: count=100, max_id=1121918108606828543, q=#gameofthrones -filter:retweets, lang=en, tweet_mode=extended, since=2019-04-01, until=2019-04-27
2019-04-28 00:38:02,951 | INFO | twitter.py:163 | wrapper | search | Requesting: count=100, max_id=1121913722916745217, q=#gameofthrones -filter:retweets, lang=en, tweet_mode=extended, since=2019-04-01, until=2019-04-27
2019-04-28 00:38:03,449 | INFO | twitter.py:163 | wrapper | search | Requesting: count=100, max_id=1121909165818089476, q=#gameofthrone

COMBINE EACH CATEGORY INTO ONE DATA FRAME

In [5]:
# got = pd.concat([got1, got2, got3, got4, got5, got6])

Save got data frame to file

In [6]:
'''

Insert code for here for saving to file to file

Remember to preserve emoji
'''
got.to_csv('got.csv')

In [7]:
'''
Insert code to read from file
tweets = pd.read('got.csv')
'''
tweets = pd.read_csv('got.csv')

COMBINE ALL DATA FRAMES INTO ONE CALLED TWEETS

In [8]:
#tweets.shape

In [9]:
got.tweet_full_text

0       Now that I saw #AvengersEndgame I can focus on...
1       LISTEN: "Power is Power" by @sza, @theweeknd a...
2       So good. So epic. And yes, whatever else there...
3       I’m not at all ready and I’ve also never been ...
4                      @Morrisons Jon Snow #GameOfThrones
5       Daenerys: *finds her people dead and her drago...
6       Inside Game of Thrones: A Story in Visual Effe...
7       Are you more excited for #AvengersEndgame or #...
8       #AvengersEndgame and #GameOfThrones battle thi...
9       Feminism in #GOT8 \n\nBrienne of Tarth, a Knig...
10      Rewatching the whole series...I feel like ever...
11      I’ve selected Tyrion Lannister to sit on the I...
12      @giffian @GameOfThrones @HBO Omg! I had to adj...
13      Meanwhile\n#GameofThrones #got #sansastark #Fo...
14      Osha: *seduces Theon's guard, slits his throat...
15      Can't wait.\n #GameofThrones https://t.co/mubK...
16      Daenarys will become the Night Queen. Heard it...
17      So man

In [10]:

# For testing - only use the trump data set and only keep the columns needed
got_tweets = pd.DataFrame(tweets, columns=['tweet_entities_mentions', 'tweet_full_text', 'tweet_entities_hashtags'])

In [11]:
got_tweets_summary = adv.extract_emoji(got_tweets['tweet_full_text'])

In [12]:
# I only want tweets where there are emojis so a comparison with and without emojis can be made
got_tweets['Number of Emojis'] = got_tweets_summary['emoji_counts']

In [13]:
got_tweets['Name of Emojis'] = got_tweets_summary['emoji_text']
got_tweets['Emojis'] = got_tweets_summary['emoji']

In [14]:
got_tweets[got_tweets['Number of Emojis'] > 0]

,tweet_entities_mentions,tweet_full_text,tweet_entities_hashtags,Number of Emojis,Name of Emojis,Emojis
2,NaN,"So good. So epic. And yes, whatever else there...","#GameOfThornes, #GameofThrones",10,"[dragon, dragon, dragon, crossed swords, dagge...","[🐉, 🐉, 🐉, ⚔️, 🗡, 👸, 🧟‍♂️, 🧙🏼‍♂️, 🧟‍♂️, 👑]"
3,NaN,I’m not at all ready and I’ve also never been ...,"#BattleOfWinterfell, #GameofThrones",1,[weary face],[😩]
7,NaN,Are you more excited for #AvengersEndgame or #...,"#AvengersEndgame, #GameOfThrones",1,[eyes],[👀]
8,NaN,#AvengersEndgame and #GameOfThrones battle thi...,"#AvengersEndgame, #GameOfThrones, #anxiety",1,[flushed face],[😳]
20,NaN,They are adorable!!❤❤ #GameofThrones https://t...,#GameofThrones,2,"[red heart, red heart]","[❤, ❤]"
29,NaN,Wi and I started watching #GameofThrones toget...,#GameofThrones,1,[thinking face],[🤔]
33,@Marvel,So I just saw #AvengersEndgame A lot of emotio...,"#AvengersEndgame, #GameofThrones",1,[grinning face with sweat],[😅]
35,NaN,OMG! So this battle is a trilogic episode? Wow...,#GameOfThrones,1,[red heart],[❤️]
46,NaN,Avengers ✅\nCan’t wait till Sunday now!!!\n\n#...,"#AvengersEndgame, #GameOfThrones",1,[check mark button],[✅]
59,NaN,When your tshirt is 100% accurate &amp; your #...,"#teacherbestie, #TeacherLife, #firstgrade, #Ga...",4,"[fire, women with bunny ears, dragon, woman te...","[🔥, 👯‍♀️, 🐉, 👩🏼‍🏫]"


In [15]:
# Only want tweets that contain emoji
got_tweets_with_emoji = got_tweets[got_tweets['Number of Emojis'] > 0]

In [16]:
got_tweets_with_emoji['tweet_full_text'] 

2       So good. So epic. And yes, whatever else there...
3       I’m not at all ready and I’ve also never been ...
7       Are you more excited for #AvengersEndgame or #...
8       #AvengersEndgame and #GameOfThrones battle thi...
20      They are adorable!!❤❤ #GameofThrones https://t...
29      Wi and I started watching #GameofThrones toget...
33      So I just saw #AvengersEndgame A lot of emotio...
35      OMG! So this battle is a trilogic episode? Wow...
46      Avengers ✅\nCan’t wait till Sunday now!!!\n\n#...
59      When your tshirt is 100% accurate &amp; your #...
62      This lady is something special. 😻#GameOfThrone...
63      Everyone in Winterfell: Gotta face the reality...
64      Am I the only one who is still more interested...
75      🔥Hot Off The Truck🔥 \n\nGoT White Walker Is Av...
78      Euron: I am the father of the baby that’s grow...
82      Wanna catch up to enjoy the most popular show'...
85      I'm soo not ready for Sundays episode of #Game...
87      And no

In [17]:
got_tweets_with_emoji['tweet_full_text'] = got_tweets_with_emoji['tweet_full_text'].str.replace(r'[^\x00-\x7F]+', '', regex=True)


In [18]:
# Find the length of each tweet - just out of curiosity
got_tweets_with_emoji['Length of Tweet'] = got_tweets_with_emoji['tweet_full_text'].apply(lambda x: len(str(x).split(" ")))

In [19]:
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
        
    return input_txt 


In [20]:
# Remove @
got_tweets_with_emoji['tweet_full_text'] = np.vectorize(remove_pattern)(got_tweets_with_emoji['tweet_full_text'], "@[\w]*")
#got_tweets_with_emoji.loc[:,"tweet_full_text"] = got_tweets_with_emoji['tweet_full_text'].apply(lambda x : " ".join(re.findall('[\w]+',x)))

In [21]:
got_tweets_with_emoji['tweet_full_text']

2       So good. So epic. And yes, whatever else there...
3       Im not at all ready and Ive also never been mo...
7       Are you more excited for #AvengersEndgame or #...
8       #AvengersEndgame and #GameOfThrones battle thi...
20      They are adorable!! #GameofThrones https://t.c...
29      Wi and I started watching #GameofThrones toget...
33      So I just saw #AvengersEndgame A lot of emotio...
35      OMG! So this battle is a trilogic episode? Wow...
46      Avengers \nCant wait till Sunday now!!!\n\n#Av...
59      When your tshirt is 100% accurate &amp; your #...
62      This lady is something special. #GameOfThrones...
63      Everyone in Winterfell: Gotta face the reality...
64      Am I the only one who is still more interested...
75      Hot Off The Truck \n\nGoT White Walker Is Avai...
78      Euron: I am the father of the baby thats growi...
82      Wanna catch up to enjoy the most popular show'...
85      I'm soo not ready for Sundays episode of #Game...
87      And no

In [22]:
# Remove URLs
got_tweets_with_emoji['tweet_full_text'] = got_tweets_with_emoji['tweet_full_text'].apply(lambda x: re.split('http:\/\/.*', str(x)))

got_tweets_with_emoji['tweet_full_text'] = got_tweets_with_emoji['tweet_full_text'].apply(lambda x: re.split('www.*', str(x))[0])

# Remove punctuation
#got_tweets_with_emoji.loc[:,"tweet_full_text"] = got_tweets_with_emoji['tweet_full_text'].apply(lambda x : " ".join(re.findall('[\w]+',x)))

In [23]:
got_tweets_with_emoji['tweet_full_text']

2       ['So good. So epic. And yes, whatever else the...
3       ['Im not at all ready and Ive also never been ...
7       ['Are you more excited for #AvengersEndgame or...
8       ['#AvengersEndgame and #GameOfThrones battle t...
20      ['They are adorable!! #GameofThrones https://t...
29      ["Wi and I started watching #GameofThrones tog...
33      ['So I just saw #AvengersEndgame A lot of emot...
35      ['OMG! So this battle is a trilogic episode? W...
46      ['Avengers \nCant wait till Sunday now!!!\n\n#...
59      ['When your tshirt is 100% accurate &amp; your...
62      ['This lady is something special. #GameOfThron...
63      ['Everyone in Winterfell: Gotta face the reali...
64      ['Am I the only one who is still more interest...
75      ['Hot Off The Truck \n\nGoT White Walker Is Av...
78      ['Euron: I am the father of the baby thats gro...
82      ["Wanna catch up to enjoy the most popular sho...
85      ["I'm soo not ready for Sundays episode of #Ga...
87      ['And 

In [24]:
# Remove numbers
got_tweets_with_emoji['tweet_full_text'] = got_tweets_with_emoji['tweet_full_text'].str.replace('\d+', '')


In [25]:
got_tweets_with_emoji['tweet_full_text']

2       ['So good. So epic. And yes, whatever else the...
3       ['Im not at all ready and Ive also never been ...
7       ['Are you more excited for #AvengersEndgame or...
8       ['#AvengersEndgame and #GameOfThrones battle t...
20      ['They are adorable!! #GameofThrones https://t...
29      ["Wi and I started watching #GameofThrones tog...
33      ['So I just saw #AvengersEndgame A lot of emot...
35      ['OMG! So this battle is a trilogic episode? W...
46      ['Avengers \nCant wait till Sunday now!!!\n\n#...
59      ['When your tshirt is % accurate &amp; your #t...
62      ['This lady is something special. #GameOfThron...
63      ['Everyone in Winterfell: Gotta face the reali...
64      ['Am I the only one who is still more interest...
75      ['Hot Off The Truck \n\nGoT White Walker Is Av...
78      ['Euron: I am the father of the baby thats gro...
82      ["Wanna catch up to enjoy the most popular sho...
85      ["I'm soo not ready for Sundays episode of #Ga...
87      ['And 

In [26]:
# Convert the tweet_full_text column to lower case 
got_tweets_with_emoji['tweet_full_text'] = got_tweets_with_emoji['tweet_full_text'].apply(lambda x: " ".join(x.lower() for x in x.split()))

In [27]:
got_tweets_with_emoji['tweet_full_text']

2       ['so good. so epic. and yes, whatever else the...
3       ['im not at all ready and ive also never been ...
7       ['are you more excited for #avengersendgame or...
8       ['#avengersendgame and #gameofthrones battle t...
20      ['they are adorable!! #gameofthrones https://t...
29      ["wi and i started watching #gameofthrones tog...
33      ['so i just saw #avengersendgame a lot of emot...
35      ['omg! so this battle is a trilogic episode? w...
46      ['avengers \ncant wait till sunday now!!!\n\n#...
59      ['when your tshirt is % accurate &amp; your #t...
62      ['this lady is something special. #gameofthron...
63      ['everyone in winterfell: gotta face the reali...
64      ['am i the only one who is still more interest...
75      ['hot off the truck \n\ngot white walker is av...
78      ['euron: i am the father of the baby thats gro...
82      ["wanna catch up to enjoy the most popular sho...
85      ["i'm soo not ready for sundays episode of #ga...
87      ['and 

In [28]:
# Import the stopwords that wil be used as part of the NLTK library
# stop words are the english version
# 
from nltk.corpus import stopwords
stop = stopwords.words('english')
got_tweets_with_emoji['tweet_full_text'] = got_tweets_with_emoji['tweet_full_text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))

In [29]:
got_tweets_with_emoji['tweet_full_text']

2       ['so good. epic. yes, whatever else is, *drago...
3       ['im ready ive also never ready #battleofwinte...
7       ['are excited #avengersendgame #gameofthrones ...
8       ['#avengersendgame #gameofthrones battle weeke...
20      ['they adorable!! #gameofthrones https://t.co/...
29      ["wi started watching #gameofthrones together,...
33      ['so saw #avengersendgame lot emotions. thanks...
35      ['omg! battle trilogic episode? wow! pull beat...
46      ['avengers \ncant wait till sunday now!!!\n\n#...
59      ['when tshirt % accurate &amp; #teacherbestie ...
62      ['this lady something special. #gameofthrones ...
63      ['everyone winterfell: gotta face reality army...
64      ['am one still interested know secret podrick ...
75      ['hot truck \n\ngot white walker available sto...
78      ['euron: father baby thats growing inside stom...
82      ["wanna catch enjoy popular show's final seaso...
85      ["i'm soo ready sundays episode #gameofthrones...
87      ['and 

In [30]:
got_tweets_with_emoji['length after prep'] = got_tweets_with_emoji['tweet_full_text'].apply(lambda x: len(str(x).split(" ")))

In [31]:
got_tweets_with_emoji

,tweet_entities_mentions,tweet_full_text,tweet_entities_hashtags,Number of Emojis,Name of Emojis,Emojis,Length of Tweet,length after prep
2,NaN,"['so good. epic. yes, whatever else is, *drago...","#GameOfThornes, #GameofThrones",10,"[dragon, dragon, dragon, crossed swords, dagge...","[🐉, 🐉, 🐉, ⚔️, 🗡, 👸, 🧟‍♂️, 🧙🏼‍♂️, 🧟‍♂️, 👑]",28,18
3,NaN,['im ready ive also never ready #battleofwinte...,"#BattleOfWinterfell, #GameofThrones",1,[weary face],[😩],18,9
7,NaN,['are excited #avengersendgame #gameofthrones ...,"#AvengersEndgame, #GameOfThrones",1,[eyes],[👀],12,6
8,NaN,['#avengersendgame #gameofthrones battle weeke...,"#AvengersEndgame, #GameOfThrones, #anxiety",1,[flushed face],[😳],18,11
20,NaN,['they adorable!! #gameofthrones https://t.co/...,#GameofThrones,2,"[red heart, red heart]","[❤, ❤]",5,4
29,NaN,"[""wi started watching #gameofthrones together,...",#GameofThrones,1,[thinking face],[🤔],43,24
33,@Marvel,['so saw #avengersendgame lot emotions. thanks...,"#AvengersEndgame, #GameofThrones",1,[grinning face with sweat],[😅],28,14
35,NaN,['omg! battle trilogic episode? wow! pull beat...,#GameOfThrones,1,[red heart],[❤️],27,15
46,NaN,['avengers \ncant wait till sunday now!!!\n\n#...,"#AvengersEndgame, #GameOfThrones",1,[check mark button],[✅],7,7
59,NaN,['when tshirt % accurate &amp; #teacherbestie ...,"#teacherbestie, #TeacherLife, #firstgrade, #Ga...",4,"[fire, women with bunny ears, dragon, woman te...","[🔥, 👯‍♀️, 🐉, 👩🏼‍🏫]",18,14


In [32]:
#import timeit
#%timeit got_tweets_with_emoji['tweet_full_text'][:700].apply(lambda x: str(TextBlob(x).correct()))

In [33]:
#got_tweets_with_emoji['tweet_full_text']

In [34]:
from textblob import Word
got_tweets_with_emoji['tweet_full_text'] = got_tweets_with_emoji['tweet_full_text'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))

In [35]:
got_tweets_with_emoji.reset_index(inplace = True) 

In [36]:
got_tweets_with_emoji = got_tweets_with_emoji.drop("index", axis=1)

In [37]:
got_tweets_with_emoji['tweet_full_text'] = got_tweets_with_emoji['tweet_full_text'].map(lambda x: x.strip())


In [38]:
got_tweets_with_emoji

,tweet_entities_mentions,tweet_full_text,tweet_entities_hashtags,Number of Emojis,Name of Emojis,Emojis,Length of Tweet,length after prep
0,NaN,"['so good. epic. yes, whatever else is, *drago...","#GameOfThornes, #GameofThrones",10,"[dragon, dragon, dragon, crossed swords, dagge...","[🐉, 🐉, 🐉, ⚔️, 🗡, 👸, 🧟‍♂️, 🧙🏼‍♂️, 🧟‍♂️, 👑]",28,18
1,NaN,['im ready ive also never ready #battleofwinte...,"#BattleOfWinterfell, #GameofThrones",1,[weary face],[😩],18,9
2,NaN,['are excited #avengersendgame #gameofthrones ...,"#AvengersEndgame, #GameOfThrones",1,[eyes],[👀],12,6
3,NaN,['#avengersendgame #gameofthrones battle weeke...,"#AvengersEndgame, #GameOfThrones, #anxiety",1,[flushed face],[😳],18,11
4,NaN,['they adorable!! #gameofthrones https://t.co/...,#GameofThrones,2,"[red heart, red heart]","[❤, ❤]",5,4
5,NaN,"[""wi started watching #gameofthrones together,...",#GameofThrones,1,[thinking face],[🤔],43,24
6,@Marvel,['so saw #avengersendgame lot emotions. thanks...,"#AvengersEndgame, #GameofThrones",1,[grinning face with sweat],[😅],28,14
7,NaN,['omg! battle trilogic episode? wow! pull beat...,#GameOfThrones,1,[red heart],[❤️],27,15
8,NaN,['avengers \ncant wait till sunday now!!!\n\n#...,"#AvengersEndgame, #GameOfThrones",1,[check mark button],[✅],7,7
9,NaN,['when tshirt % accurate &amp; #teacherbestie ...,"#teacherbestie, #TeacherLife, #firstgrade, #Ga...",4,"[fire, women with bunny ears, dragon, woman te...","[🔥, 👯‍♀️, 🐉, 👩🏼‍🏫]",18,14


In [39]:
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
# Function to apply lemmatization to a list of words
def words_lemmatizer(text, encoding="utf8"):
    words = nltk.word_tokenize(text)
    lemma_words = []
    wl = WordNetLemmatizer()
    for word in words:
        pos = find_pos(word)
        lemma_words.append(wl.lemmatize(word, pos).encode(encoding))
    return " ".join(lemma_words)

In [40]:
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()

In [41]:
def lemmatize_text(text):
    return [lemmatizer.lemmatize(w) for w in w_tokenizer.tokenize(text)]

In [42]:
got_tweets_with_emoji.tweet_full_text

0      ['so good. epic. yes, whatever else is, *drago...
1      ['im ready ive also never ready #battleofwinte...
2      ['are excited #avengersendgame #gameofthrones ...
3      ['#avengersendgame #gameofthrones battle weeke...
4      ['they adorable!! #gameofthrones https://t.co/...
5      ["wi started watching #gameofthrones together,...
6      ['so saw #avengersendgame lot emotions. thanks...
7      ['omg! battle trilogic episode? wow! pull beat...
8      ['avengers \ncant wait till sunday now!!!\n\n#...
9      ['when tshirt % accurate &amp; #teacherbestie ...
10     ['this lady something special. #gameofthrones ...
11     ['everyone winterfell: gotta face reality army...
12     ['am one still interested know secret podrick ...
13     ['hot truck \n\ngot white walker available sto...
14     ['euron: father baby thats growing inside stom...
15     ["wanna catch enjoy popular show's final seaso...
16     ["i'm soo ready sunday episode #gameofthrones!...
17     ['and this!! #gameofthro

In [43]:
got_tweets_with_emoji.tweet_full_text.apply(lemmatize_text)


0      [['so, good., epic., yes,, whatever, else, is,...
1      [['im, ready, ive, also, never, ready, #battle...
2      [['are, excited, #avengersendgame, #gameofthro...
3      [['#avengersendgame, #gameofthrones, battle, w...
4      [['they, adorable!!, #gameofthrones, https://t...
5      [["wi, started, watching, #gameofthrones, toge...
6      [['so, saw, #avengersendgame, lot, emotions., ...
7      [['omg!, battle, trilogic, episode?, wow!, pul...
8      [['avengers, \ncant, wait, till, sunday, now!!...
9      [['when, tshirt, %, accurate, &amp;, #teacherb...
10     [['this, lady, something, special., #gameofthr...
11     [['everyone, winterfell:, gotta, face, reality...
12     [['am, one, still, interested, know, secret, p...
13     [['hot, truck, \n\ngot, white, walker, availab...
14     [['euron:, father, baby, thats, growing, insid...
15     [["wanna, catch, enjoy, popular, show's, final...
16     [["i'm, soo, ready, sunday, episode, #gameofth...
17     [['and, this!!, #gameoft

In [44]:
new_got = got_tweets_with_emoji.head(10)

In [45]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [46]:
# Tokenization
tokens = []
# Lemmatization
lemma = []
# Part of Speech
pos = []

for doc in nlp.pipe(got_tweets_with_emoji['tweet_full_text'].astype('unicode').values, batch_size=50,
                        n_threads=3):
    if doc.is_parsed:
        tokens.append([n.text for n in doc])
        lemma.append([n.lemma_ for n in doc])
        pos.append([n.pos_ for n in doc])
    else:
        # We want to make sure that the lists of parsed results have the
        # same number of entries of the original Dataframe, so add some blanks in case the parse fails
        tokens.append(None)
        lemma.append(None)
        pos.append(None)

got_tweets_with_emoji['got_tokens'] = tokens
got_tweets_with_emoji['got_lemma'] = lemma
got_tweets_with_emoji['got_pos'] = pos

In [47]:
got_tweets_with_emoji
#word_summary.keys()

,tweet_entities_mentions,tweet_full_text,tweet_entities_hashtags,Number of Emojis,Name of Emojis,Emojis,Length of Tweet,length after prep,got_tokens,got_lemma,got_pos
0,NaN,"['so good. epic. yes, whatever else is, *drago...","#GameOfThornes, #GameofThrones",10,"[dragon, dragon, dragon, crossed swords, dagge...","[🐉, 🐉, 🐉, ⚔️, 🗡, 👸, 🧟‍♂️, 🧙🏼‍♂️, 🧟‍♂️, 👑]",28,18,"[[, ', so, good, ., epic, ., yes, ,, whatever,...","[[, ', so, good, ., epic, ., yes, ,, whatever,...","[PUNCT, PUNCT, ADV, ADJ, PUNCT, ADJ, PUNCT, IN..."
1,NaN,['im ready ive also never ready #battleofwinte...,"#BattleOfWinterfell, #GameofThrones",1,[weary face],[😩],18,9,"[[, ', i, m, ready, i, ve, also, never, ready,...","[[, ', -PRON-, be, ready, -PRON-, have, also, ...","[PUNCT, PUNCT, PRON, VERB, ADJ, PRON, VERB, AD..."
2,NaN,['are excited #avengersendgame #gameofthrones ...,"#AvengersEndgame, #GameOfThrones",1,[eyes],[👀],12,6,"[[, ', are, excited, #, avengersendgame, #, ga...","[[, ', be, excite, #, avengersendgame, #, game...","[PUNCT, PUNCT, VERB, VERB, SYM, NOUN, ADJ, NOU..."
3,NaN,['#avengersendgame #gameofthrones battle weeke...,"#AvengersEndgame, #GameOfThrones, #anxiety",1,[flushed face],[😳],18,11,"[[, ', #, avengersendgame, #, gameofthrones, b...","[[, ', #, avengersendgame, #, gameofthrone, ba...","[PUNCT, PUNCT, SYM, NOUN, SYM, NOUN, NOUN, NOU..."
4,NaN,['they adorable!! #gameofthrones https://t.co/...,#GameofThrones,2,"[red heart, red heart]","[❤, ❤]",5,4,"[[, ', they, adorable, !, !, #, gameofthrones,...","[[, ', -PRON-, adorable, !, !, #, gameofthrone...","[PUNCT, PUNCT, PRON, VERB, PUNCT, PUNCT, ADJ, ..."
5,NaN,"[""wi started watching #gameofthrones together,...",#GameofThrones,1,[thinking face],[🤔],43,24,"[[, "", wi, started, watching, #, gameofthrones...","[[, "", wi, start, watch, #, gameofthrone, toge...","[PUNCT, PUNCT, PRON, VERB, VERB, SYM, NOUN, AD..."
6,@Marvel,['so saw #avengersendgame lot emotions. thanks...,"#AvengersEndgame, #GameofThrones",1,[grinning face with sweat],[😅],28,14,"[[, ', so, saw, #, avengersendgame, lot, emoti...","[[, ', so, see, #, avengersendgame, lot, emoti...","[PUNCT, PUNCT, ADV, VERB, SYM, NOUN, NOUN, NOU..."
7,NaN,['omg! battle trilogic episode? wow! pull beat...,#GameOfThrones,1,[red heart],[❤️],27,15,"[[, ', omg, !, battle, trilogic, episode, ?, w...","[[, ', omg, !, battle, trilogic, episode, ?, w...","[PUNCT, PUNCT, NOUN, PUNCT, NOUN, NOUN, NOUN, ..."
8,NaN,['avengers \ncant wait till sunday now!!!\n\n#...,"#AvengersEndgame, #GameOfThrones",1,[check mark button],[✅],7,7,"[[, ', avengers, \ncant, wait, till, sunday, n...","[[, ', avenger, \ncant, wait, till, sunday, no...","[PUNCT, PUNCT, NOUN, PRON, VERB, ADP, NOUN, VE..."
9,NaN,['when tshirt % accurate &amp; #teacherbestie ...,"#teacherbestie, #TeacherLife, #firstgrade, #Ga...",4,"[fire, women with bunny ears, dragon, woman te...","[🔥, 👯‍♀️, 🐉, 👩🏼‍🏫]",18,14,"[[, ', when, tshirt, %, accurate, &, amp, ;, #...","[[, ', when, tshirt, %, accurate, &, amp, ;, #...","[PUNCT, PUNCT, ADV, NOUN, NOUN, ADJ, CCONJ, NO..."


In [48]:
#trump_tweets_with_emoji['tweet_full_text'].apply(lambda x: TextBlob(x).sentiment)
new_got['Orig Tweet Sent'] = new_got['tweet_full_text'].apply(lambda x: TextBlob(x).sentiment[0] )

In [49]:
new_got

,tweet_entities_mentions,tweet_full_text,tweet_entities_hashtags,Number of Emojis,Name of Emojis,Emojis,Length of Tweet,length after prep,Orig Tweet Sent
0,NaN,"['so good. epic. yes, whatever else is, *drago...","#GameOfThornes, #GameofThrones",10,"[dragon, dragon, dragon, crossed swords, dagge...","[🐉, 🐉, 🐉, ⚔️, 🗡, 👸, 🧟‍♂️, 🧙🏼‍♂️, 🧟‍♂️, 👑]",28,18,0.333333
1,NaN,['im ready ive also never ready #battleofwinte...,"#BattleOfWinterfell, #GameofThrones",1,[weary face],[😩],18,9,0.050000
2,NaN,['are excited #avengersendgame #gameofthrones ...,"#AvengersEndgame, #GameOfThrones",1,[eyes],[👀],12,6,0.375000
3,NaN,['#avengersendgame #gameofthrones battle weeke...,"#AvengersEndgame, #GameOfThrones, #anxiety",1,[flushed face],[😳],18,11,0.100000
4,NaN,['they adorable!! #gameofthrones https://t.co/...,#GameofThrones,2,"[red heart, red heart]","[❤, ❤]",5,4,0.781250
5,NaN,"[""wi started watching #gameofthrones together,...",#GameofThrones,1,[thinking face],[🤔],43,24,0.000000
6,@Marvel,['so saw #avengersendgame lot emotions. thanks...,"#AvengersEndgame, #GameofThrones",1,[grinning face with sweat],[😅],28,14,0.300000
7,NaN,['omg! battle trilogic episode? wow! pull beat...,#GameOfThrones,1,[red heart],[❤️],27,15,0.062500
8,NaN,['avengers \ncant wait till sunday now!!!\n\n#...,"#AvengersEndgame, #GameOfThrones",1,[check mark button],[✅],7,7,0.000000
9,NaN,['when tshirt % accurate &amp; #teacherbestie ...,"#teacherbestie, #TeacherLife, #firstgrade, #Ga...",4,"[fire, women with bunny ears, dragon, woman te...","[🔥, 👯‍♀️, 🐉, 👩🏼‍🏫]",18,14,0.200000


In [50]:
#new_got['result'] = new_got['tweet_full_text'].map(lambda x: x.strip('+-').strip('aAbBcC'))
#new_got['result'] = new_got['tweet_full_text'].str.replace('[a-z]','')
new_got['result'] = new_got['tweet_full_text'].str.replace('[#]','')

In [51]:
new_got.sort_values('Orig Tweet Sent')

,tweet_entities_mentions,tweet_full_text,tweet_entities_hashtags,Number of Emojis,Name of Emojis,Emojis,Length of Tweet,length after prep,Orig Tweet Sent,result
5,NaN,"[""wi started watching #gameofthrones together,...",#GameofThrones,1,[thinking face],[🤔],43,24,0.000000,"[""wi started watching gameofthrones together, ..."
8,NaN,['avengers \ncant wait till sunday now!!!\n\n#...,"#AvengersEndgame, #GameOfThrones",1,[check mark button],[✅],7,7,0.000000,['avengers \ncant wait till sunday now!!!\n\na...
1,NaN,['im ready ive also never ready #battleofwinte...,"#BattleOfWinterfell, #GameofThrones",1,[weary face],[😩],18,9,0.050000,['im ready ive also never ready battleofwinter...
7,NaN,['omg! battle trilogic episode? wow! pull beat...,#GameOfThrones,1,[red heart],[❤️],27,15,0.062500,['omg! battle trilogic episode? wow! pull beat...
3,NaN,['#avengersendgame #gameofthrones battle weeke...,"#AvengersEndgame, #GameOfThrones, #anxiety",1,[flushed face],[😳],18,11,0.100000,['avengersendgame gameofthrones battle weekend...
9,NaN,['when tshirt % accurate &amp; #teacherbestie ...,"#teacherbestie, #TeacherLife, #firstgrade, #Ga...",4,"[fire, women with bunny ears, dragon, woman te...","[🔥, 👯‍♀️, 🐉, 👩🏼‍🏫]",18,14,0.200000,['when tshirt % accurate &amp; teacherbestie t...
6,@Marvel,['so saw #avengersendgame lot emotions. thanks...,"#AvengersEndgame, #GameofThrones",1,[grinning face with sweat],[😅],28,14,0.300000,['so saw avengersendgame lot emotions. thanks ...
0,NaN,"['so good. epic. yes, whatever else is, *drago...","#GameOfThornes, #GameofThrones",10,"[dragon, dragon, dragon, crossed swords, dagge...","[🐉, 🐉, 🐉, ⚔️, 🗡, 👸, 🧟‍♂️, 🧙🏼‍♂️, 🧟‍♂️, 👑]",28,18,0.333333,"['so good. epic. yes, whatever else is, *drago..."
2,NaN,['are excited #avengersendgame #gameofthrones ...,"#AvengersEndgame, #GameOfThrones",1,[eyes],[👀],12,6,0.375000,['are excited avengersendgame gameofthrones we...
4,NaN,['they adorable!! #gameofthrones https://t.co/...,#GameofThrones,2,"[red heart, red heart]","[❤, ❤]",5,4,0.781250,['they adorable!! gameofthrones https://t.co/e...


In [74]:
def l(l):
    return l[0]

new_got['Name of Emojis'].apply(l) 

0                      dragon
1                  weary face
2                        eyes
3                flushed face
4                   red heart
5               thinking face
6    grinning face with sweat
7                   red heart
8           check mark button
9                        fire
Name: Name of Emojis, dtype: object

Emoji Sentement Analysis

In [52]:
emo = pd.read_csv('3258168_1699780384_emojiranking.csv')

In [53]:
emo = pd.DataFrame(emo, columns=['Char', 'Sentiment score', 'Unicode name'])

In [54]:
emo['Unicode name'] = emo['Unicode name'].str.lower()

In [124]:
emo = emo.iloc[1:]
emo.head()

,Char,Sentiment score,Unicode name
7,😊,0.644,smiling face with smiling eyes
8,👌,0.563,ok hand sign
9,💕,0.632,two hearts
10,👏,0.52,clapping hands sign
11,😁,0.449,grinning face with smiling eyes


In [86]:
'''

Enter code here for emoji sent analysis

Sudo code:

If the tweets.tweet__text_full containts any of the emojis in emo.Char or Unicode name

add the emo.Semtiment Score to the orignal sentiment score and add to a new column in the orifinal data frame

Save data to file - again preserving emoji

'''


'0.449'

In [125]:
#using name
def emofinder(e):
    
    for l in e:
        if l in emo['Unicode name'].tolist():
    #             print(emo['Unicode name'])
            return emo['Sentiment score'][emo['Unicode name'].tolist().index(l)]
            break
        continue

#using emoji
def emofinder1(e):
    
    for l in e:
        if l in emo['Char'].tolist():
    #             print(emo['Unicode name'])
            return emo['Sentiment score'][emo['Char'].tolist().index(l)]
            break
        continue
        

#applying
new_got['Sentiment Score']= new_got['Name of Emojis'].apply(emofinder)
new_got['Sentiment Score']= new_got['Emojis'].apply(emofinder1)

In [126]:
# new_got['Sentiment Score']

In [127]:
new_got

,tweet_entities_mentions,tweet_full_text,tweet_entities_hashtags,Number of Emojis,Name of Emojis,Emojis,Length of Tweet,length after prep,Orig Tweet Sent,result,Sentiment Score
0,NaN,"['so good. epic. yes, whatever else is, *drago...","#GameOfThornes, #GameofThrones",10,"[dragon, dragon, dragon, crossed swords, dagge...","[🐉, 🐉, 🐉, ⚔️, 🗡, 👸, 🧟‍♂️, 🧙🏼‍♂️, 🧟‍♂️, 👑]",28,18,0.333333,"['so good. epic. yes, whatever else is, *drago...",0.154
1,NaN,['im ready ive also never ready #battleofwinte...,"#BattleOfWinterfell, #GameofThrones",1,[weary face],[😩],18,9,0.050000,['im ready ive also never ready battleofwinter...,0.563
2,NaN,['are excited #avengersendgame #gameofthrones ...,"#AvengersEndgame, #GameOfThrones",1,[eyes],[👀],12,6,0.375000,['are excited avengersendgame gameofthrones we...,0.178
3,NaN,['#avengersendgame #gameofthrones battle weeke...,"#AvengersEndgame, #GameOfThrones, #anxiety",1,[flushed face],[😳],18,11,0.100000,['avengersendgame gameofthrones battle weekend...,0.738
4,NaN,['they adorable!! #gameofthrones https://t.co/...,#GameofThrones,2,"[red heart, red heart]","[❤, ❤]",5,4,0.781250,['they adorable!! gameofthrones https://t.co/e...,None
5,NaN,"[""wi started watching #gameofthrones together,...",#GameofThrones,1,[thinking face],[🤔],43,24,0.000000,"[""wi started watching gameofthrones together, ...",None
6,@Marvel,['so saw #avengersendgame lot emotions. thanks...,"#AvengersEndgame, #GameofThrones",1,[grinning face with sweat],[😅],28,14,0.300000,['so saw avengersendgame lot emotions. thanks ...,0.287
7,NaN,['omg! battle trilogic episode? wow! pull beat...,#GameOfThrones,1,[red heart],[❤️],27,15,0.062500,['omg! battle trilogic episode? wow! pull beat...,None
8,NaN,['avengers \ncant wait till sunday now!!!\n\n#...,"#AvengersEndgame, #GameOfThrones",1,[check mark button],[✅],7,7,0.000000,['avengers \ncant wait till sunday now!!!\n\na...,-0.232
9,NaN,['when tshirt % accurate &amp; #teacherbestie ...,"#teacherbestie, #TeacherLife, #firstgrade, #Ga...",4,"[fire, women with bunny ears, dragon, woman te...","[🔥, 👯‍♀️, 🐉, 👩🏼‍🏫]",18,14,0.200000,['when tshirt % accurate &amp; teacherbestie t...,0.691


In [128]:
new_got.to_csv('again preserving emoji.csv')